In [1]:
from bs4 import BeautifulSoup 
import requests 
import re
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [4]:
folder_path = "Authors/"
names = pd.read_csv(folder_path + "Authors2024.csv")

In [6]:
folder_path = "Works/"
papers = pd.read_csv(folder_path + "IC2S2_papers.csv")

In [74]:
def pair_exists(df, author_a, author_b):
    mask = ((df["author_1"] == author_a) & (df["author_2"] == author_b)) | \
           ((df["author_1"] == author_b) & (df["author_2"] == author_a))
    match_index = df[mask].index
    return match_index, mask.any()  # Returns True if the pair exists

In [ ]:
edge_dict = {}
for authors in papers["author_ids"].values:
    cleaned_string = authors.replace("[", "").replace("]", "").replace("'", "")
    authors = np.array(cleaned_string.split(", "))
    
    for i,author1 in enumerate(authors):
        for j in range(i+1,len(authors)):
            author2 =    authors[j]
            author_pair = tuple(sorted([author1,author2]))
            if author_pair in edge_dict:
                edge_dict[author_pair] += 1
            else:
                edge_dict[author_pair] = 1
df = pd.DataFrame([{"author_1": pair[0], "author_2": pair[1], "weight": weight} for pair, weight in edge_dict.items()])
df

,author_1,author_2,weight
0,https://openalex.org/A5010373001,https://openalex.org/A5072156055,1
1,https://openalex.org/A5032475130,https://openalex.org/A5072156055,1
2,https://openalex.org/A5010373001,https://openalex.org/A5032475130,1
3,https://openalex.org/A5054630728,https://openalex.org/A5072156055,2
4,https://openalex.org/A5005839111,https://openalex.org/A5072156055,4
...,...,...,...
46178,https://openalex.org/A5067122850,https://openalex.org/A5090329397,1
46179,https://openalex.org/A5088656844,https://openalex.org/A5090329397,1
46180,https://openalex.org/A5067122850,https://openalex.org/A5088656844,1
46181,https://openalex.org/A5067122850,https://openalex.org/A5103326255,1


In [ ]:
#df.to_csv("Authors/author_edgelist.csv", index = False)

In [2]:
df = pd.read_csv("Authors/author_edgelist.csv")

In [ ]:
G = nx.Graph()

G.add_nodes_from(df['author_1'].unique())
G.add_nodes_from(df['author_2'].unique())

edges = [(row['author_1'], row['author_2'], row['weight']) for index, row in df.iterrows()]

G.add_weighted_edges_from(edges)

EdgeView([('https://openalex.org/A5010373001', 'https://openalex.org/A5072156055'), ('https://openalex.org/A5010373001', 'https://openalex.org/A5032475130'), ('https://openalex.org/A5032475130', 'https://openalex.org/A5072156055'), ('https://openalex.org/A5054630728', 'https://openalex.org/A5072156055'), ('https://openalex.org/A5054630728', 'https://openalex.org/A5014221568'), ('https://openalex.org/A5054630728', 'https://openalex.org/A5026969792'), ('https://openalex.org/A5054630728', 'https://openalex.org/A5035199834'), ('https://openalex.org/A5054630728', 'https://openalex.org/A5070873457'), ('https://openalex.org/A5054630728', 'https://openalex.org/A5027733964'), ('https://openalex.org/A5054630728', 'https://openalex.org/A5061206863'), ('https://openalex.org/A5054630728', 'https://openalex.org/A5042787477'), ('https://openalex.org/A5054630728', 'https://openalex.org/A5044130443'), ('https://openalex.org/A5054630728', 'https://openalex.org/A5084395089'), ('https://openalex.org/A5054